# Tensorflow with Estimators

As we saw previously how to build a full Multi-Layer Perceptron model with full Sessions in Tensorflow. Unfortunately this was an extremely involved process. However developers have created Estimators that have an easier to use flow!

It is much easier to use, but you sacrifice some level of customization of your model. Let's go ahead and explore it!

## Get the Data

We will the iris data set.

Let's get the data:

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('iris.csv')

In [3]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [4]:
df.columns

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)', 'target'],
      dtype='object')

In [5]:
df.columns = ['sepal_length', 'sepal_width', 'petal_length',
       'petal_width', 'target']

In [6]:
df['target'] = df['target'].apply(int)

In [7]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [8]:
X = df.drop('target', axis=1)
y = df['target']

## Train Test Split

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

# Estimators

Let's show you how to use the simpler Estimator interface!

In [11]:
import tensorflow as tf

## Feature Columns

In [12]:
X.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], dtype='object')

In [13]:
feat_cols = []

for col in X.columns:
    feat_cols.append(tf.feature_column.numeric_column(col))

In [14]:
feat_cols

[_NumericColumn(key='sepal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='sepal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='petal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='petal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

## Input Function

In [15]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, batch_size=10, num_epochs=5, shuffle=True)

In [16]:
classifier = tf.estimator.DNNClassifier(hidden_units=[10,20,10], n_classes=3, feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/2x/16whkbcd3bd0x2f2rtl5phvc0000gn/T/tmp0df1zmqs', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a166a4ba8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [17]:
classifier.train(input_fn=input_func, steps=50)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/2x/16whkbcd3bd0x2f2rtl5phvc0000gn/T/tmp0df1zmqs/model.ckpt.
INFO:tensorflow:loss = 11.6461315, step = 1
INFO:tensorflow:Saving checkpoints for 50 into /var/folders/2x/16whkbcd3bd0x2f2rtl5phvc0000gn/T/tmp0df1zmqs/model.ckpt.
INFO:tensorflow:Loss for final step: 2.0888724.


## Model Evaluation

** Use the predict method from the classifier model to create predictions from X_test **

In [18]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test, batch_size=len(X_test), shuffle=False)

In [19]:
note_predictions = list(classifier.predict(input_fn=pred_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/2x/16whkbcd3bd0x2f2rtl5phvc0000gn/T/tmp0df1zmqs/model.ckpt-50
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [21]:
note_predictions[0]

{'logits': array([ 1.1436495, -3.3773859, -6.2542677], dtype=float32),
 'probabilities': array([9.8864031e-01, 1.0754189e-02, 6.0556899e-04], dtype=float32),
 'class_ids': array([0]),
 'classes': array([b'0'], dtype=object)}

In [23]:
final_preds = []

for pred in note_predictions:
    final_preds.append(pred['class_ids'][0])

** Now create a classification report and a Confusion Matrix. Does anything stand out to you?**

In [24]:
from sklearn.metrics import classification_report,confusion_matrix

In [25]:
print('Confusion_matrix: \n')
print(confusion_matrix(y_test, final_preds))

Confusion_matrix: 

[[20  0  0]
 [ 0 12  3]
 [ 0  0 10]]


In [26]:
print('Classification_report: \n')
print(classification_report(y_test, final_preds))

Classification_report: 

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      0.80      0.89        15
           2       0.77      1.00      0.87        10

   micro avg       0.93      0.93      0.93        45
   macro avg       0.92      0.93      0.92        45
weighted avg       0.95      0.93      0.93        45



# Great Job!